## Problem:
The problem is that LWC and NBC are too low by a factor of 2.
The origin of this issue is not yet clear!

## Solution:
simply multiply the scale factor of the parameter dataset by ``2``!!

In [ ]:
import xarray as xr
from pathlib import Path
from sdm_eurec4a import RepositoryPath

RepoPaths = RepositoryPath("levante")

### Data loading

In [ ]:
OBS_data_dir = RepoPaths.data_dir
input_path = OBS_data_dir / Path("model/input_v4.0/")
assert input_path.is_dir()
output_path = OBS_data_dir / Path("model/input_v4.1/")
output_path.mkdir(exist_ok=True, parents=False)

psd_name = "particle_size_distribution_parameters.nc"
potential_temperature_name = "potential_temperature_parameters.nc"
relative_humidity_name = "relative_humidity_parameters.nc"
pressure_name = "pressure_parameters.nc"
names = [psd_name, potential_temperature_name, relative_humidity_name, pressure_name]
# copy all these file to the new directory

### Copy and adjust datasets if necessary

In [ ]:
for name in names:
    ds = xr.open_dataset(input_path / Path(name)).to_netcdf(output_path / Path(name))
# modify the psd file

ds_parameters = xr.open_dataset(input_path / Path(psd_name))


ds_parameters["scale_factor1"] = 2 * ds_parameters["scale_factor1"]
ds_parameters["scale_factor2"] = 2 * ds_parameters["scale_factor2"]

ds_parameters.to_netcdf(output_path / Path(psd_name))

### Validate the datasets are the same.
nans in div are due to 0 / 0 calculation.
The diff in scale factors is expected, BUT the div needs to be 2!

In [ ]:
for name in names:
    print("------")
    print(name)
    ds_parameters = xr.open_dataset(input_path / Path(name))
    ds_parameters_new = xr.open_dataset(output_path / Path(name))

    print(
        "".ljust(15),
        "div mean".ljust(10),
        "div std".ljust(10),
        "diff mean".ljust(10),
        "diff std".ljust(10),
    )
    for var in ds_parameters.variables:
        m = f"{((ds_parameters_new[var]/ ds_parameters[var]).mean().values):.2e}".ljust(10)
        s = f"{((ds_parameters_new[var]/ ds_parameters[var]).std().values):.2e}".ljust(10)
        dm = f"{((ds_parameters_new[var] - ds_parameters[var]).mean().values):.2e}".ljust(10)
        ds = f"{((ds_parameters_new[var] - ds_parameters[var]).std().values):.2e}".ljust(10)

        print(var.ljust(15), m, s, dm, ds)